In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy

from pathlib import Path

nlp = spacy.load("en_core_web_sm")

In [2]:
df = pd.read_json("./results/multinli_transforms_bertscore_dev_matched.jsonl", lines=True).set_index('pairID')
df['sentence1_transform'] = [row['sentence1_transform1'] if row['sentence1_transform1_bertscore'] > row['sentence1_transform2_bertscore'] else row['sentence1_transform2'] for index, row in df.iterrows()]
df['sentence2_transform'] = [row['sentence2_transform1'] if row['sentence2_transform1_bertscore'] > row['sentence2_transform2_bertscore'] else row['sentence2_transform2'] for index, row in df.iterrows()]
df.head()    

,annotator_labels,genre,gold_label,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,...,sentence1_transform1,sentence1_transform2,sentence2_transform1,sentence2_transform2,sentence1_transform1_bertscore,sentence1_transform2_bertscore,sentence2_transform1_bertscore,sentence2_transform2_bertscore,sentence1_transform,sentence2_transform
pairID,,,,,,,,,,,,,,,,,,,,,
63735n,"[neutral, entailment, neutral, neutral, neutral]",slate,neutral,63735,The new rights are nice enough,( ( The ( new rights ) ) ( are ( nice enough )...,(ROOT (S (NP (DT The) (JJ new) (NNS rights)) (...,Everyone really likes the newest benefits,( Everyone ( really ( likes ( the ( newest ben...,(ROOT (S (NP (NN Everyone)) (VP (ADVP (RB real...,...,"Arrr, thar be some fine new rights fer us!","'Tis a pleasant sight, these new rights, me he...","Arrr, me hearties be lovin' the latest booty.",All hands on deck be happy with the brand new ...,0.835507,0.846157,0.847767,0.878438,"'Tis a pleasant sight, these new rights, me he...",All hands on deck be happy with the brand new ...
91383c,"[contradiction, contradiction, contradiction, ...",government,contradiction,91383,This site includes a list of all award winners...,( ( This site ) ( ( includes ( ( ( ( a list ) ...,(ROOT (S (NP (DT This) (NN site)) (VP (VBZ inc...,The Government Executive articles housed on th...,( ( ( The ( Government ( Executive articles ) ...,(ROOT (S (NP (NP (DT The) (NNP Government) (NN...,...,This site be includin' a list o' all award win...,"Arrr, this here site be havin' a list o' all t...",Ye Government Executive articles housed on tha...,Thar Government Executive pieces housed on th'...,0.938552,0.870573,0.949286,0.898377,This site be includin' a list o' all award win...,Ye Government Executive articles housed on tha...
755e,"[entailment, entailment, entailment, entailmen...",telephone,entailment,755,uh i don't know i i have mixed emotions about ...,( ( ( ( uh ( i ( ( do n't ) ( know ( ( i i ) (...,(ROOT (SINV (S (S (INTJ (UH uh)) (NP (FW i)) (...,"I like him for the most part, but would still ...",( I ( ( ( ( ( ( like him ) ( for ( the ( most ...,(ROOT (S (NP (PRP I)) (VP (VP (VBP like) (NP (...,...,"Arrr, I be not sure me hearties, sometimes I f...","Aye, me hearty, me feelings be mixed about him...","I be likin' him for the most part, but I'd sti...","I be fancyin' him for the most part, but I'd s...",0.873443,0.876103,0.920837,0.929856,"Aye, me hearty, me feelings be mixed about him...","I be fancyin' him for the most part, but I'd s..."
78013c,"[contradiction, contradiction, contradiction, ...",telephone,contradiction,78013,yeah i i think my favorite restaurant is alway...,( yeah ( ( i i ) ( think ( ( my ( favorite res...,(ROOT (S (VP (VB yeah) (NP (NP (FW i) (FW i)) ...,My favorite restaurants are always at least a ...,( ( My ( favorite restaurants ) ) ( ( ( ( are ...,(ROOT (S (NP (PRP$ My) (JJ favorite) (NNS rest...,...,"Aye, me favorite galley be always the one near...","Arr, methinks me top tavern's always been the ...",Me favorite taverns be always at least a hundr...,My preferred grub spots be a hundred leagues o...,0.869231,0.845367,0.933945,0.901702,"Aye, me favorite galley be always the one near...",Me favorite taverns be always at least a hundr...
96377c,"[contradiction, contradiction, contradiction, ...",telephone,contradiction,96377,i don't know um do you do a lot of camping,( i ( ( do n't ) ( know ( um ( do ( you ( do (...,(ROOT (S (NP (FW i)) (VP (VBP do) (RB n't) (VP...,I know exactly.,( I ( ( know exactly ) . ) ),(ROOT (S (NP (PRP I)) (VP (VBP know) (ADVP (RB...,...,"I don't be knowin' matey, do ye partake in a l...","Arrr, I be clueless, but do yer fancy a lot o'...",I savvy precisely.,I be knowin' precisely.,0.874817,0.852997,0.909285,0.882909,"I don't be knowin' matey, do ye partake in a l...",I savvy precisely.


In [3]:
for x, y in df.groupby('style'):
    prems_tokenized = [[token.lower_ for token in nlp(row['sentence1_transform']) if not token.is_punct] for index, row in y.iterrows()]
    hypos_tokenized = [[token.lower_ for token in nlp(row['sentence2_transform']) if not token.is_punct] for index, row in y.iterrows()]
    
    different_words = set([token for sent in prems_tokenized + hypos_tokenized for token in sent])

    avg_prem_len = sum([len(prem) for prem in prems_tokenized]) / len(prems_tokenized)
    avg_hypo_len = sum([len(hypo) for hypo in hypos_tokenized]) / len(hypos_tokenized)
    avg_word_overlap = sum([len(set(prem) & set(hypo)) / len(set(hypo)) for prem, hypo in zip(prems_tokenized, hypos_tokenized)]) / len(prems_tokenized)
    
    print(x)
    print(f"Average premise length TF: {avg_prem_len} tokens")
    print(f"Average hypothesis length TF: {avg_hypo_len} tokens")
    print(f"Average word overlap TF: {avg_word_overlap}")
    print(f"Different words TF: {len(different_words)}")
    print()

emb
Average premise length TF: 30.91741741741742 tokens
Average hypothesis length TF: 19.578978978978977 tokens
Average word overlap TF: 0.295174781747641
Different words TF: 16929

Average premise length: 20.294894894894895 tokens
Average hypothesis length: 10.176876876876877 tokens
Average word overlap: 0.4859014084440292
Different words: 10132

pir
Average premise length TF: 20.811094452773613 tokens
Average hypothesis length TF: 11.856971514242879 tokens
Average word overlap TF: 0.3573219236048372
Different words TF: 11677

Average premise length: 19.811094452773613 tokens
Average hypothesis length: 10.24047976011994 tokens
Average word overlap: 0.48514607470214527
Different words: 10027

sha
Average premise length TF: 21.05577211394303 tokens
Average hypothesis length TF: 12.166116941529236 tokens
Average word overlap TF: 0.33321089075314864
Different words TF: 13563

Average premise length: 19.837481259370314 tokens
Average hypothesis length: 10.084257871064468 tokens
Average wor

In [9]:
df_sampled = df.sample(3333)

prems_tokenized = [[token.lower_ for token in nlp(row['sentence1']) if not token.is_punct] for index, row in df_sampled.iterrows()]
hypos_tokenized = [[token.lower_ for token in nlp(row['sentence2']) if not token.is_punct] for index, row in df_sampled.iterrows()]

different_words = set([token for sent in prems_tokenized + hypos_tokenized for token in sent])

avg_prem_len = sum([len(prem) for prem in prems_tokenized]) / len(prems_tokenized)
avg_hypo_len = sum([len(hypo) for hypo in hypos_tokenized]) / len(hypos_tokenized)
avg_word_overlap = sum([len(set(prem) & set(hypo)) / len(set(hypo)) for prem, hypo in zip(prems_tokenized, hypos_tokenized)]) / len(prems_tokenized)

print(f"Average premise length: {avg_prem_len} tokens")
print(f"Average hypothesis length: {avg_hypo_len} tokens")
print(f"Average word overlap: {avg_word_overlap}")
print(f"Different words: {len(different_words)}")

Average premise length: 19.74977497749775 tokens
Average hypothesis length: 10.125112511251125 tokens
Average word overlap: 0.49457985743246413
Different words: 9893
